In [ ]:
!pip install tfx

In [24]:
import os
import numpy as np 
import pandas as pd
import pprint
pp = pprint.PrettyPrinter()
from typing import Dict, List, Text

import tensorflow as tf
import tensorflow_model_analysis as tfma

import tfx
from tfx.components import CsvExampleGen
from tfx.components import StatisticsGen
from tfx.components import SchemaGen
from tfx.components import ExampleValidator
from tfx.components import Transform
from tfx.components import Trainer
from tfx.components import Evaluator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing
from tfx.utils.dsl_utils import external_input

Checking the library versions. 

In [3]:
print('Tensorflow version:',tf.__version__)
print('TFX version',tfx.__version__)

Tensorflow version: 2.4.1
TFX version 0.28.0


Removing any previous data.

In [4]:
!rm -rf data.*
!rm -rf *trainer.py
!sudo rm -r /content/tfx

TFX requires a certain folder structure which is created as shown below. Content is the directory used to access storage space on the Google Drive. 

In [5]:
! cd /content/
! mkdir /content/tfx
! mkdir /content/tfx/pipelines
! mkdir /content/tfx/metadata
! mkdir /content/tfx/logs
! mkdir /content/tfx/data
! mkdir /content/tfx/serving_model

### Download example data

We use the [Taxi Trips dataset]((https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew) ) released by City of Chicago in the TFX pipeline. 

In [6]:
!wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv

--2021-04-05 05:34:26--  https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1922812 (1.8M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.83M  --.-KB/s    in 0.06s   

2021-04-05 05:34:26 (28.5 MB/s) - ‘data.csv’ saved [1922812/1922812]



If you look at the folder structure on the left, you can see the dataset is downloaded. 

In [7]:
data = pd.read_csv('/content/data.csv')
data.head()

,pickup_community_area,fare,trip_start_month,trip_start_hour,trip_start_day,trip_start_timestamp,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,trip_miles,pickup_census_tract,dropoff_census_tract,payment_type,company,trip_seconds,dropoff_community_area,tips
0,NaN,12.45,5,19,6,1400269500,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Credit Card,Chicago Elite Cab Corp. (Chicago Carriag,0.0,NaN,0.0
1,NaN,0.00,3,19,5,1362683700,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Unknown,Chicago Elite Cab Corp.,300.0,NaN,0.0
2,60.0,27.05,10,2,3,1380593700,41.836150,-87.648788,NaN,NaN,12.6,NaN,NaN,Cash,Taxi Affiliation Services,1380.0,NaN,0.0
3,10.0,5.85,10,1,2,1382319000,41.985015,-87.804532,NaN,NaN,0.0,NaN,NaN,Cash,Taxi Affiliation Services,180.0,NaN,0.0
4,14.0,16.65,5,7,5,1369897200,41.968069,-87.721559,NaN,NaN,0.0,NaN,NaN,Cash,Dispatch Taxi Affiliation,1080.0,NaN,0.0


In [8]:
#drop useless columns
data = data.drop(columns=['trip_start_timestamp','trip_miles','pickup_census_tract',
              'dropoff_census_tract','trip_seconds','payment_type','tips', 
              'company','dropoff_community_area','pickup_community_area'])

#check for null values
data.isnull().sum()

fare                   0
trip_start_month       0
trip_start_hour        0
trip_start_day         0
pickup_latitude        2
pickup_longitude       2
dropoff_latitude     483
dropoff_longitude    483
dtype: int64

In [9]:
#drop rows with null values
data = data.dropna()

#Splitting the data into train and validation data
np.random.seed(seed=2)
mask = np.random.rand(len(data)) < 0.9
train_data = data[mask]
eval_data = data[~mask]

print(len(train_data))
print(len(eval_data))

#Saving data to train and test csvs for TFX
train_data.to_csv("/content/tfx/data/train_data.csv", index=False, header=True)
eval_data.to_csv("/content/tfx/data/eval_data.csv",index=False, header=True)

13077
1442


### Set up paths for pipeline

In [10]:
#defining all paths 

_tfx_root = os.path.join(os.getcwd(), 'tfx')
_pipeline_root = os.path.join(_tfx_root, 'pipelines')
_metadata_db_root = os.path.join(_tfx_root, 'metadata.db')
_log_root = os.path.join(_tfx_root, 'model')
_model_root = os.path.join(_tfx_root, 'model')
_data_root = os.path.join(_tfx_root, 'data')
_serving_model_dir = os.path.join(_tfx_root, 'serving_model')
_data_filepath = os.path.join(_data_root, "train_data.csv")

#creating python files that will be used later
_input_fn_module_file = 'inputfn_trainer.py'
_constants_module_file = 'constants_trainer.py'
_model_trainer_module_file = 'model_trainer.py'

### Create InteractiveContext

This allows us to run TFX components interactively within the notebook

In [12]:
"""The InteractiveContext is created using default parameters. It will use a temporary directory with an ML Metadata database instance 
To use your own pipeline root/database pass the optional parameters _pipeline_root and metadata_connection_config can be specified in the call to 
Interactive Context. 
"""

context = InteractiveContext(pipeline_root=_tfx_root)

## Run TFX components interactively

We create each TFX component one by one and visualise their artifacts. Artifacts refer to the output of the components. 

### Example Gen

The `ExampleGen` component is used for data ingestion. It is the first step in the TFX pipeline. 
It will:

1. **Split** the data into training and evaluation sets (2/3 and 1/3 respectively)
2. Convert the data into `tf.Example` format
3. **Copy the splits** into the `_tfx_root` directory for other components to access. 

`ExampleGen` takes the path to the data source as input, i.e `_data_root` which contains the downloaded CSV file. 

Note: Here we instantiate components one by one and run them with `InteractiveContext.run()`. In a production seeting, all components are sprecified upfront in a `Pipeline` and passed to the orchestrator ([Building a TFX Pipeline Guide](../tfx/guide/build_tfx_pipeline))). 

In [14]:
example_gen = CsvExampleGen(input=external_input(_data_root))
context.run(example_gen)

Instructions for updating:
external_input is deprecated, directly pass the uri to ExampleGen.


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/content/tfx/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:2,total_bytes:1006907,xor_checksum:0,sum_checksum:3235201770"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

The `ExampleGen` component outputs two artifacts - the training and evaluation examples. 

In [16]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /content/tfx/CsvExampleGen/examples/1


Taking a look at the first three examples:

The following code takes the data from training data from the folder `train` under ``CsvExampleGen, reads the `TfRecord` files from memory and picks the first three record. 

In [25]:
#Get the URI of the output artifact representing the training examples, which is a directory
train_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'train')

#Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri,name) for name in os.listdir(train_uri)]

#Create a `TfRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

#Iterate over first 3 records, decode them by converting to mupy format and parsing
for tfrecord in dataset.take(3):
  serialized_example = tfrecord.numpy()
  example = tf.train.Example()
  example.ParseFromString(serialized_example)
  pp.pprint(example)

features {
  feature {
    key: "dropoff_latitude"
    value {
      float_list {
        value: 41.92045211791992
      }
    }
  }
  feature {
    key: "dropoff_longitude"
    value {
      float_list {
        value: -87.6799545288086
      }
    }
  }
  feature {
    key: "fare"
    value {
      float_list {
        value: 3.8499999046325684
      }
    }
  }
  feature {
    key: "pickup_latitude"
    value {
      float_list {
        value: 41.8996696472168
      }
    }
  }
  feature {
    key: "pickup_longitude"
    value {
      float_list {
        value: -87.66983795166016
      }
    }
  }
  feature {
    key: "trip_start_day"
    value {
      int64_list {
        value: 6
      }
    }
  }
  feature {
    key: "trip_start_hour"
    value {
      int64_list {
        value: 15
      }
    }
  }
  feature {
    key: "trip_start_month"
    value {
      int64_list {
        value: 3
      }
    }
  }
}

features {
  feature {
    key: "dropoff_latitude"
    value {
      fl

Each record/datapoint has 8 columns and has the column name, datatype and value for feach feature. 

### Statisics Gen

This component **computes the statistics** of the dataset such as the mean, standard deviation, minimum and maximum values in the dataset. 

`StatisticsGen` takes as input the dataset ingested using `ExampleGen`. 

In [27]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/content/tfx/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

The input and output of the `StatisticsGen` component can be clearly seen above. The `InteractiveContext` is used to see the statistics generated by the `StatisticsGen` component for both the train and eval data using **TFDV**. 

In [28]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


This is a useful feature as we can see all the statistics for both the train and eval sets such as the number of missing values, distribution of the data for the dataset in one place without the use of additonal libraries and plots. 

### Schema Gen

The next step is to generate a schema from the data statistics using the `SchemaGen` component. This schema takes as input the statistics generated with `StatisticsGen` and defines the expected range of values, data types and properties of the feature in the dataset. It also uses the [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) library. 

Note: The generated schema only tries to infer the basis properties of the data. 

In [29]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/content/tfx/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

Visualizing the generated schema using the `InteractiveContext`

In [30]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'dropoff_latitude',FLOAT,required,single,-
'dropoff_longitude',FLOAT,required,single,-
'fare',FLOAT,required,single,-
'pickup_latitude',FLOAT,required,single,-
'pickup_longitude',FLOAT,required,single,-
'trip_start_day',INT,required,single,-
'trip_start_hour',INT,required,single,-
'trip_start_month',INT,required,single,-


All the columns/features present in the dataset are shown in the schema along with its properties. 
The purpose of a schema is twofold:
1. Identify if retraining is required by examining the changes in the data (missing/additional columns, changes in values of data)
2. It checks the latest data to make sure all the required features are present in the dataset during retraining


### Example Validator

This component detects anomalies in the dataset based on the statistics and schema generated.
It also uses the [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) library.


In [31]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/content/tfx/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

The inputs to the `ExampleValidator` are the statistics and schema and the output is the anomalies found by the data. 

Visualizing the anomalies as a table:

In [32]:
context.show(example_validator.outputs['anomalies'])

/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:188: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


No anomalies are found, which is expected as the schema of the eval dataset is tailored to the train dataset. This schema can be used to
1. Detect anomalies 
2. Debug model performance
3. Understand how the data evolves over time

### Transform

Instead of using Apache Beam and Tensorflow Transform, constants are calculated using Pandas/Numpy. The pickup and drop pff latitude are binned using `qcut` function in pandas which chooses the bins such that the same number of records are present in each bin. 

In [35]:
bins_lat = pd.qcut(list(data['dropoff_latitude'].values) + list(data['pickup_latitude'].values), q=20, duplicates='drop', retbins=True)[1]
bins_lon = pd.qcut(list(data['dropoff_longitude'].values) + list(data['pickup_longitude'].values), q=20, duplicates='drop', retbins=True)[1]

In [36]:
code = '''
bins_lat = {bins_lat}
bins_lon = {bins_lon}
'''

code = code.replace('{bins_lat}', str(list(bins_lat)))
code = code.replace('{bins_lon}', str(list(bins_lon)))

with open(_constants_module_file, 'w') as writefile:
    writefile.write(code)

### Trainer
This component trains a model defined in Ternsorflow. The `Trainer` component uses the schema from `SchemaGen` and the transformed data and graph from `Transform`. It also taked the training parameters and a model that contains the user-defined code for the model

In [38]:
%%writefile {_input_fn_module_file}

import os
import tensorflow as tf

###############################
##Feature engineering functions
def feature_engg_features(features):
  #Add new features
  features['distance'] = ((features['pickup_latitude'] - features['dropoff_latitude'])**2 +  (features['pickup_longitude'] - features['dropoff_longitude'])**2)**0.5
  features['trip_start_month'] = tf.strings.as_string(features['trip_start_month'])
  features['trip_start_hour'] = tf.strings.as_string(features['trip_start_hour'])
  features['trip_start_day'] = tf.strings.as_string(features['trip_start_day'])

  return(features)

#To be called from TF
def feature_engg(features, label):
  #Add new features
  features = feature_engg_features(features)

  return(features, label)

def make_input_fn(dir_uri, mode, vnum_epochs = None, batch_size = 512):
    def decode_tfr(serialized_example):
      # 1. define a parser
      features = tf.io.parse_example(
        serialized_example,
        # Defaults are not specified since both keys are required.
        features={
            'dropoff_latitude': tf.io.FixedLenFeature([], tf.float32),
            'dropoff_longitude': tf.io.FixedLenFeature([], tf.float32),
            'fare': tf.io.FixedLenFeature([], tf.float32),
            'pickup_latitude': tf.io.FixedLenFeature([], tf.float32, default_value = 0.0),
            'pickup_longitude': tf.io.FixedLenFeature([], tf.float32, default_value = 0.0),
            'trip_start_day': tf.io.FixedLenFeature([], tf.int64),
            'trip_start_hour': tf.io.FixedLenFeature([], tf.int64),
            'trip_start_month': tf.io.FixedLenFeature([], tf.int64)
        })

      return features, features['fare']

    def _input_fn(v_test=False):
      # Get the list of files in this directory (all compressed TFRecord files)
      tfrecord_filenames = tf.io.gfile.glob(dir_uri)

      # Create a `TFRecordDataset` to read these files
      dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

      if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = vnum_epochs # indefinitely
      else:
        num_epochs = 1 # end-of-input after this

      dataset = dataset.batch(batch_size)
      dataset = dataset.prefetch(buffer_size = batch_size)

      #Convert TFRecord data to dict
      dataset = dataset.map(decode_tfr)

      #Feature engineering
      dataset = dataset.map(feature_engg)

      if mode == tf.estimator.ModeKeys.TRAIN:
          num_epochs = vnum_epochs # indefinitely
          dataset = dataset.shuffle(buffer_size = batch_size)
      else:
          num_epochs = 1 # end-of-input after this

      dataset = dataset.repeat(num_epochs)       
      
      #Begins - Uncomment for testing only -----------------------------------------------------<
      if v_test == True:
        print(next(dataset.__iter__()))
        
      #End - Uncomment for testing only -----------------------------------------------------<
      return dataset
    return _input_fn

Overwriting inputfn_trainer.py


In [39]:
##Test the input function
import inputfn_trainer as ift

#Test dataset read + Feat Engg function's - output's CSV + Feature engg columns
eval_file = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'eval/*')
fn_d = ift.make_input_fn(dir_uri = eval_file,
                    mode = tf.estimator.ModeKeys.EVAL,
                    # vnum_epochs = 1,
                    batch_size = 10)

fn_d(v_test=True)

({'dropoff_latitude': <tf.Tensor: shape=(10,), dtype=float32, numpy=
array([41.920452, 41.920452, 41.849247, 42.00157 , 42.00157 , 42.00157 ,
       42.00157 , 42.00157 , 41.97244 , 41.89251 ], dtype=float32)>, 'dropoff_longitude': <tf.Tensor: shape=(10,), dtype=float32, numpy=
array([-87.679955, -87.679955, -87.62414 , -87.695015, -87.695015,
       -87.695015, -87.695015, -87.695015, -87.67111 , -87.62621 ],
      dtype=float32)>, 'fare': <tf.Tensor: shape=(10,), dtype=float32, numpy=
array([12.25,  6.05,  7.85,  7.45,  3.45,  9.85,  9.45,  3.25, 12.25,
        6.45], dtype=float32)>, 'pickup_latitude': <tf.Tensor: shape=(10,), dtype=float32, numpy=
array([41.952824, 41.906025, 41.871017, 41.97517 , 42.00157 , 41.968067,
       41.97517 , 42.00157 , 41.90838 , 41.89266 ], dtype=float32)>, 'pickup_longitude': <tf.Tensor: shape=(10,), dtype=float32, numpy=
array([-87.653244, -87.67531 , -87.63141 , -87.687515, -87.695015,
       -87.72156 , -87.687515, -87.695015, -87.670944, -87.65253

<RepeatDataset shapes: ({dropoff_latitude: (None,), dropoff_longitude: (None,), fare: (None,), pickup_latitude: (None,), pickup_longitude: (None,), trip_start_day: (None,), trip_start_hour: (None,), trip_start_month: (None,), distance: (None,)}, (None,)), types: ({dropoff_latitude: tf.float32, dropoff_longitude: tf.float32, fare: tf.float32, pickup_latitude: tf.float32, pickup_longitude: tf.float32, trip_start_day: tf.string, trip_start_hour: tf.string, trip_start_month: tf.string, distance: tf.float32}, tf.float32)>

In [52]:
%%writefile {_model_trainer_module_file}

import tensorflow as tf
import tensorflow.keras as keras
import inputfn_trainer as ift
import constants_trainer as ct

from tfx.components.trainer.fn_args_utils import FnArgs
print(tf.__version__)

device = "gpu"

if device == "tpu":
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.config.experimental_connect_to_cluster(resolver)
  # This is the TPU initialization code that has to be at the beginning.
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
else:
  strategy = tf.distribute.MultiWorkerMirroredStrategy()

#Create model
params_default = {
    'lr' : 0.001,
    'beta_1' : 0.99,
    'beta_2' : 0.999,
    'epsilon' : 1e-08,
    'decay' : 0.01,
    'hidden_layers' : 1
}

# Define feature columns(Including feature engineered ones )
# These are the features which come from the TF Data pipeline
def create_feature_cols():
    #Keras format features
    k_month = tf.keras.Input(name='trip_start_month', shape=(1,), dtype=tf.string)
    k_hour  = tf.keras.Input(name='trip_start_hour', shape=(1,), dtype=tf.string)
    k_day  = tf.keras.Input(name='trip_start_day', shape=(1,), dtype=tf.string)
    k_picklat  = tf.keras.Input(name='pickup_latitude', shape=(1,), dtype=tf.float32)
    k_picklon  = tf.keras.Input(name='pickup_longitude', shape=(1,), dtype=tf.float32)
    k_droplat  = tf.keras.Input(name='dropoff_latitude', shape=(1,), dtype=tf.float32)
    k_droplon  = tf.keras.Input(name='dropoff_longitude', shape=(1,), dtype=tf.float32)
    k_distance  = tf.keras.Input(name='distance', shape=(1,), dtype=tf.float32)
    keras_dict_input = {'trip_start_month': k_month, 'trip_start_hour': k_hour, 'trip_start_day' : k_day,
                        'pickup_latitude': k_picklat, 'pickup_longitude': k_picklon,
                        'dropoff_latitude': k_droplat, 'dropoff_longitude': k_droplon, 'distance' : k_distance
                        }

    return({'K' : keras_dict_input})

def create_keras_model(feature_cols, bins_lat, bins_lon,  params = params_default):
    METRICS = [
            keras.metrics.RootMeanSquaredError(name='rmse')
    ]

    #Input layers
    input_feats = []
    for inp in feature_cols['K'].keys():
      input_feats.append(feature_cols['K'][inp])

    ##Input processing
    ##https://keras.io/examples/structured_data/structured_data_classification_from_scratch/
    ##https://github.com/tensorflow/community/blob/master/rfcs/20191212-keras-categorical-inputs.md

    ##Handle categorical attributes( One-hot encoding )
    cat_day = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=['1','2','3','4','5','6','7'], mask_token=None)(feature_cols['K']['trip_start_day'])
    cat_day = tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=7)(cat_day)

    cat_hour = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=['1','2','3','4','5','6','7','8'
                                                                                      '9','10','11','12','13','14','15','16',
                                                                                      '17','18','19','20','21','22','23','0'
                                                                                      ], mask_token=None)(feature_cols['K']['trip_start_hour'])
    cat_hour = tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=24)(cat_hour)

    cat_month = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=['1','2','3','4','5','6','7','8'
                                                                                      '9','10','11','12'], mask_token=None)(feature_cols['K']['trip_start_month'])
    cat_month = tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=12)(cat_month)

    # cat_company = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=df['company'].unique(), mask_token=None)(feature_cols['K']['company'])
    # cat_company = tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens=len(df['company'].unique()))(cat_company)

    ##Binning
    bins_pickup_lat = tf.keras.layers.experimental.preprocessing.Discretization(bins = bins_lat)(feature_cols['K']['pickup_latitude'])
    cat_pickup_lat = tf.keras.layers.experimental.preprocessing.CategoryEncoding(len(bins_lat)+1)(bins_pickup_lat)

    bins_pickup_lon = tf.keras.layers.experimental.preprocessing.Discretization(bins = bins_lon)(feature_cols['K']['pickup_longitude'])
    cat_pickup_lon = tf.keras.layers.experimental.preprocessing.CategoryEncoding(len(bins_lon)+1)(bins_pickup_lon)

    bins_drop_lat = tf.keras.layers.experimental.preprocessing.Discretization(bins = bins_lat)(feature_cols['K']['dropoff_latitude'])
    cat_drop_lat = tf.keras.layers.experimental.preprocessing.CategoryEncoding(len(bins_lat)+1)(bins_drop_lat)

    bins_drop_lon = tf.keras.layers.experimental.preprocessing.Discretization(bins = bins_lon)(feature_cols['K']['dropoff_longitude'])
    cat_drop_lon = tf.keras.layers.experimental.preprocessing.CategoryEncoding(len(bins_lon)+1)(bins_drop_lon)

    ##Categorical cross
    cross_day_hour = tf.keras.layers.experimental.preprocessing.CategoryCrossing()([cat_day, cat_hour])
    hash_cross_day_hour = tf.keras.layers.experimental.preprocessing.Hashing(num_bins=24 * 7)(cross_day_hour)
    cat_cross_day_hour = tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens = 24* 7)(hash_cross_day_hour)

    cross_pick_lon_lat = tf.keras.layers.experimental.preprocessing.CategoryCrossing()([cat_pickup_lat, cat_pickup_lon])
    hash_cross_pick_lon_lat = tf.keras.layers.experimental.preprocessing.Hashing(num_bins=(len(bins_lat) + 1) ** 2)(cross_pick_lon_lat)

    cross_drop_lon_lat = tf.keras.layers.experimental.preprocessing.CategoryCrossing()([cat_drop_lat, cat_drop_lon])
    hash_cross_drop_lon_lat = tf.keras.layers.experimental.preprocessing.Hashing(num_bins=(len(bins_lat) + 1) ** 2)(cross_drop_lon_lat)

    # Cross to embedding
    embed_cross_pick_lon_lat = tf.keras.layers.Embedding(((len(bins_lat) + 1) ** 2), 4)(hash_cross_pick_lon_lat)
    embed_cross_pick_lon_lat = tf.reduce_sum(embed_cross_pick_lon_lat, axis=-2)

    embed_cross_drop_lon_lat = tf.keras.layers.Embedding(((len(bins_lat) + 1) ** 2), 4)(hash_cross_drop_lon_lat)
    embed_cross_drop_lon_lat = tf.reduce_sum(embed_cross_drop_lon_lat, axis=-2)

    # Also pass time attributes as Deep signal( Cast to integer )
    int_trip_start_day = tf.strings.to_number(feature_cols['K']['trip_start_day'], tf.float32)
    int_trip_start_hour = tf.strings.to_number(feature_cols['K']['trip_start_hour'], tf.float32)
    int_trip_start_month = tf.strings.to_number(feature_cols['K']['trip_start_month'], tf.float32)

    #Add feature engineered columns - LAMBDA layer

    ###Create MODEL
    ####Concatenate all features( Numerical input )
    x_input_numeric = tf.keras.layers.concatenate([
                    feature_cols['K']['pickup_latitude'], feature_cols['K']['pickup_longitude'],
                    feature_cols['K']['dropoff_latitude'], feature_cols['K']['dropoff_longitude'],
                    feature_cols['K']['distance'], embed_cross_pick_lon_lat, embed_cross_drop_lon_lat,
                    int_trip_start_day, int_trip_start_hour, int_trip_start_month
                    ])

    #DEEP - This Dense layer connects to input layer - Numeric Data
    x_numeric = tf.keras.layers.Dense(32, activation='relu', kernel_initializer="he_uniform")(x_input_numeric)
    x_numeric = tf.keras.layers.BatchNormalization()(x_numeric)

    ####Concatenate all Categorical features( Categorical converted )
    x_input_categ = tf.keras.layers.concatenate([
                    cat_month, cat_cross_day_hour, cat_pickup_lat, cat_pickup_lon,
                    cat_drop_lat, cat_drop_lon
                    ])
    
    #WIDE - This Dense layer connects to input layer - Categorical Data
    x_categ = tf.keras.layers.Dense(32, activation='relu', kernel_initializer="he_uniform")(x_input_categ)

    ####Concatenate both Wide and Deep layers
    x = tf.keras.layers.concatenate([x_categ, x_numeric])

    for l_ in range(params['hidden_layers']):
        x = tf.keras.layers.Dense(32, activation='relu', kernel_initializer="he_uniform",
                                  activity_regularizer=tf.keras.regularizers.l2(0.00001))(x)
        x = tf.keras.layers.BatchNormalization()(x)

    #Final Layer
    out = tf.keras.layers.Dense(1, activation='relu')(x)
    model = tf.keras.Model(input_feats, out)

    #Set optimizer
    opt = tf.keras.optimizers.Adam(lr= params['lr'], beta_1=params['beta_1'], 
                                        beta_2=params['beta_2'], epsilon=params['epsilon'])

    #Compile model
    model.compile(loss='mean_squared_error',  optimizer=opt, metrics = METRICS)

    #Print Summary
    print(model.summary())
    return model

def keras_train_and_evaluate(model, train_dataset, validation_dataset, epochs=100):
  #Add callbacks
  reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=5, min_lr=0.00001, verbose = 1)
  
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")

  #Train and Evaluate
  out = model.fit(train_dataset, 
                  validation_data = validation_dataset,
                  epochs=epochs,
                  # validation_steps = 3,   ###Keep this none for running evaluation on full EVAL data every epoch
                  steps_per_epoch = 100,   ###Has to be passed - Cant help it :) [ Number of batches per epoch ]
                  callbacks=[reduce_lr, #modelsave_callback, #tensorboard_callback, 
                             keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True, verbose=True)]
                  )

  return model

def save_model(model, model_save_path):
  @tf.function
  def serving(dropoff_latitude, dropoff_longitude, pickup_latitude, pickup_longitude, trip_start_day, trip_start_hour, trip_start_month):
      ##Feature engineering( calculate distance )
      distance = tf.cast( tf.sqrt((tf.abs(dropoff_latitude - pickup_latitude))**2 + (tf.abs(dropoff_longitude - pickup_longitude))**2), tf.float32)

      payload = {
          'dropoff_latitude': dropoff_latitude,
          'dropoff_longitude': dropoff_longitude,
          'pickup_latitude': pickup_latitude,
          'pickup_longitude': pickup_longitude,
          'trip_start_day': trip_start_day,
          'trip_start_hour': trip_start_hour,
          'trip_start_month': trip_start_month,
          'distance': distance
      }
      
      ## Predict
      ##IF THERE IS AN ERROR IN NUMBER OF PARAMS PASSED HERE OR DATA TYPE THEN IT GIVES ERROR, "COULDN'T COMPUTE OUTPUT TENSOR"
      predictions = model(payload)
      return predictions

  serving = serving.get_concrete_function(trip_start_day=tf.TensorSpec([None,], dtype= tf.string, name='trip_start_day'), 
                                          trip_start_hour=tf.TensorSpec([None,], dtype= tf.string, name='trip_start_hour'),
                                          trip_start_month=tf.TensorSpec([None], dtype= tf.string, name='trip_start_month'), 
                                          dropoff_latitude=tf.TensorSpec([None,], dtype= tf.float32, name='dropoff_latitude'),
                                          dropoff_longitude=tf.TensorSpec([None,], dtype= tf.float32, name='dropoff_longitude'), 
                                          pickup_latitude=tf.TensorSpec([None,], dtype= tf.float32, name='pickup_latitude'),
                                          pickup_longitude=tf.TensorSpec([None,], dtype= tf.float32, name='pickup_longitude')
                                          )

  # version = "1"  #{'serving_default': call_output}
  tf.saved_model.save(
      model,
      model_save_path + "/",
      signatures=serving
  )

##Main function called by TFX
def run_fn(fn_args: FnArgs):
  #Create dataset input functions
  train_dataset = ift.make_input_fn(dir_uri = fn_args.train_files,
                      mode = tf.estimator.ModeKeys.TRAIN,
                      batch_size = 128)()

  validation_dataset = ift.make_input_fn(dir_uri = fn_args.eval_files,
                      mode = tf.estimator.ModeKeys.EVAL,
                      batch_size = 512)()

  #Create model
  m_ = create_keras_model(params = params_default, feature_cols = create_feature_cols(),
                          bins_lat = ct.bins_lat,
                          bins_lon = ct.bins_lon)
  tf.keras.utils.plot_model(m_, show_shapes=True, rankdir="LR")

  #Train model
  m_ = keras_train_and_evaluate(m_, train_dataset, validation_dataset, fn_args.custom_config['epochs'])

  #Save model with custom signature
  save_model(m_, fn_args.serving_model_dir)

Overwriting model_trainer.py


In [53]:
trainer = Trainer(
    module_file=os.path.abspath(_model_trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=example_gen.outputs['examples'],
    train_args=trainer_pb2.TrainArgs(),
    eval_args=trainer_pb2.EvalArgs(),
    custom_config=({'epochs':1})
)
context.run(trainer)

2.4.1


INFO:tensorflow:Using MirroredStrategy with devices ('/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/device:CPU:0',)


INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO


INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO


...

100/100 [==============================] - ETA: 0s - loss: 231.3719 - rmse: 15.1552WARNING:tensorflow:Using a while_loop for converting BoostedTreesBucketize


100/100 [==============================] - 20s 173ms/step - loss: 231.4957 - rmse: 15.1598 - val_loss: 108.3223 - val_rmse: 10.4075


INFO:tensorflow:Assets written to: /content/tfx/Trainer/model/10/serving_model_dir/assets


INFO:tensorflow:Assets written to: /content/tfx/Trainer/model/10/serving_model_dir/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 10
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 15
        type_id: 13
        uri: "/content/tfx/Trainer/model/10"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 13
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 16
        type_id: 14
        uri: "/content/tfx/Trainer/model_run/10"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

### Analyze Training with TensorBoard

The trainer artifact points to a directory that contains the model subdirectories. 

In [57]:

model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'serving_model_dir')
pp.pprint(os.listdir(model_dir))

['serving_model_dir']
['variables', 'saved_model.pb', 'assets']


### Pusher

The `Pusher` component checks if a model has passed validation and if yes, exports the model to `_serving_model_dir`. 

In [60]:
pusher = Pusher(
    model=trainer.outputs['model'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))

context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 11
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 17
        type_id: 16
        uri: "/content/tfx/Pusher/pushed_model/11"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "/content/tfx/serving_model/1617616277"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1617616277"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "PushedModel"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [61]:
pusher.outputs

{'pushed_model': Channel(
    type_name: PushedModel
    artifacts: [Artifact(artifact: id: 17
type_id: 16
uri: "/content/tfx/Pusher/pushed_model/11"
custom_properties {
  key: "name"
  value {
    string_value: "pushed_model"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Pusher"
  }
}
custom_properties {
  key: "pushed"
  value {
    int_value: 1
  }
}
custom_properties {
  key: "pushed_destination"
  value {
    string_value: "/content/tfx/serving_model/1617616277"
  }
}
custom_properties {
  key: "pushed_version"
  value {
    string_value: "1617616277"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 16
name: "PushedModel"
)]
    additional_properties: {}
    additional_custom_properties: {}
)}

In [62]:
push_uri = pusher.outputs.pushed_model.get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
  pp.pprint(item)

('serving_default',
 <ConcreteFunction signature_wrapper(*, trip_start_month, trip_start_day, pickup_longitude, trip_start_hour, dropoff_latitude, dropoff_longitude, pickup_latitude) at 0x7F1C477767D0>)


### **Full Pipeline**


In [63]:
!rm -rf data.*
!rm -rf *.csv
!sudo rm -r /content/tfx

! cd /content/
! mkdir /content/tfx/
! mkdir /content/tfx/pipelines
! mkdir /content/tfx/metadata
! mkdir /content/tfx/logs
! mkdir /content/tfx/data
! mkdir /content/tfx/serving_model

! mkdir /content/train_data/
! mkdir /content/eval_data/

!wget https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv

--2021-04-05 10:00:15--  https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/chicago_taxi_pipeline/data/simple/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1922812 (1.8M) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]   1.83M  --.-KB/s    in 0.06s   

2021-04-05 10:00:15 (28.7 MB/s) - ‘data.csv’ saved [1922812/1922812]



In [64]:
data = pd.read_csv('/content/data.csv')

##Drop useless columns
data = data.drop(['trip_start_timestamp','trip_miles','pickup_census_tract',
              'dropoff_census_tract','trip_seconds','payment_type','tips', 
              'company','dropoff_community_area','pickup_community_area'], axis=1)

#Drop NA rows
data = data.dropna()

##Keep a test set for final testing( TFX internally splits train and validation data )
np.random.seed(seed=2)
msk = np.random.rand(len(data)) < 0.9
traindf = data[msk]
evaldf = data[~msk]

print(len(traindf))
print(len(evaldf))

traindf.to_csv("/content/train_data/data.csv", index=False, header=True)
evaldf.to_csv("/content/eval_data/eval.csv", index=False, header=False)

13077
1442


In [65]:
def create_final_pipeline(
    pipeline_name: Text,
    root_path: Text,
    data_path,
    training_params: Dict[Text, Text],
) -> pipeline.Pipeline:

  _pipeline_root = os.path.join(root_path, 'pipelines');      # Join ~/tfx/pipelines/
  _metadata_db_root = os.path.join(root_path, 'metadata.db');    # Join ~/tfx/metadata.db
  _log_root = os.path.join(root_path, 'logs');
  _model_root = os.path.join(root_path, 'model');
  _serving_model_dir = os.path.join(root_path, 'serving_model')

    # Full pipeline
  example_gen = CsvExampleGen(input=external_input(data_path))

  statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

  infer_schema = SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=False)

  validate_stats = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=infer_schema.outputs['schema'])

  trainer = Trainer(
      module_file=os.path.abspath(_model_trainer_module_file),
      custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
      examples=example_gen.outputs['examples'],
      train_args=trainer_pb2.TrainArgs(),
      eval_args=trainer_pb2.EvalArgs(),
      custom_config=(training_params)
      )

  pusher = Pusher(
      model=trainer.outputs['model'],
      push_destination=pusher_pb2.PushDestination(
          filesystem=pusher_pb2.PushDestination.Filesystem(
              base_directory=_serving_model_dir)))

  # This pipeline obj carries the business logic of the pipeline, but no runner-specific information
  # was included.
  return pipeline.Pipeline(
    pipeline_name=  pipeline_name,
    pipeline_root=  root_path,
    components=[
        example_gen, statistics_gen, infer_schema, validate_stats,
        trainer, pusher
    ],
    # metadata_connection_config = metadata.sqlite_metadata_connection_config(_metadata_db_root),
    metadata_connection_config = metadata.sqlite_metadata_connection_config(_metadata_db_root),
    enable_cache=True,
    beam_pipeline_args=['--direct_num_workers=%d' % 0],
  )

In [66]:
#Run pipeline locally
from tfx.orchestration.local.local_dag_runner import LocalDagRunner

##Define all paths
_tfx_root = os.path.join(os.getcwd(), 'tfx')

#Config params
training_params = {"epochs": 50}

#Create and run pipeline
p_ = create_final_pipeline(root_path = _tfx_root, 
                           pipeline_name="local_pipeline", 
                           data_path="/content/train_data",
                           training_params=training_params)

LocalDagRunner().run(p_)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
pickup_latitude (InputLayer)    [(None, 1)]          0                                            
__________________________________________________________________________________________________
pickup_longitude (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
dropoff_latitude (InputLayer)   [(None, 1)]          0                                            
__________________________________________________________________________________________________
dropoff_longitude (InputLayer)  [(None, 1)]          0                                            
____________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/functional.py:595: UserWarning: Input dict contained keys ['fare'] which did not match any model input. They will be ignored by the model.
  [n for n in tensors.keys() if n not in ref_input_names])


100/100 [==============================] - ETA: 0s - loss: 201.3877 - rmse: 14.1251WARNING:tensorflow:Using a while_loop for converting BoostedTreesBucketize


100/100 [==============================] - 19s 171ms/step - loss: 201.5415 - rmse: 14.1311 - val_loss: 105.1480 - val_rmse: 10.2539
Epoch 2/50
100/100 [==============================] - 16s 160ms/step - loss: 221.1786 - rmse: 14.8197 - val_loss: 298.1359 - val_rmse: 17.2666
Epoch 3/50
100/100 [==============================] - 16s 160ms/step - loss: 145.1671 - rmse: 11.9938 - val_loss: 380.8831 - val_rmse: 19.5162
Epoch 4/50
100/100 [==============================] - 16s 158ms/step - loss: 150.8071 - rmse: 12.1376 - val_loss: 229.3057 - val_rmse: 15.1428
Epoch 5/50
100/100 [==============================] - 16s 159ms/step - loss: 97.8387 - rmse: 9.8421 - val_loss: 165.4669 - val_rmse: 12.8634
Epoch 6/50
100/100 [==============================] - 16s 157ms/step - loss: 109.8315 - rmse: 10.2414 - val_loss: 132.7436 - val_rmse: 11.5214

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
Epoch 7/50
100/100 [==============================] - 16s 158ms/step - lo

INFO:tensorflow:Assets written to: /content/tfx/Trainer/model/3/serving_model_dir/assets


INFO:tensorflow:Assets written to: /content/tfx/Trainer/model/3/serving_model_dir/assets


In [67]:
!saved_model_cli show --dir "/content/tfx/Pusher/pushed_model/4" --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['dropoff_latitude'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: serving_default_dropoff_latitude:0
    inputs['dropoff_longitude'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: serving_default_dropoff_longitude:0
    inputs['pickup_latitude'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
        name: serving_default_pickup_latitude:0
    inputs['pickup_longitude'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1)
 

In [68]:
#LOCAL: Predict using Keras prediction function
saved_mod = tf.saved_model.load("/content/tfx/Pusher/pushed_model/4")

#Get prediction function from serving
f = saved_mod.signatures['serving_default']

#Run prediction function from serving
f(dropoff_latitude=tf.convert_to_tensor([41.920452]), dropoff_longitude = tf.convert_to_tensor([-87.679955]), pickup_latitude = tf.convert_to_tensor([41.952823]), 
  pickup_longitude =tf.convert_to_tensor([-87.653244]), trip_start_day=tf.convert_to_tensor(["1"]), trip_start_hour=tf.convert_to_tensor(["5"]),
  trip_start_month=tf.convert_to_tensor(["6"]))

{'output_0': <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[5.428641]], dtype=float32)>}